# HW 1: Corporate Bond Pricing (due by 9.20 Fri)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2018`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2018/HW2/HW2.ipynb`__  (Please use the same name for repository and ipynb file)
* Adding more code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [2]:
import numpy as np

In [3]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10
F = 1 
c = 0

In [119]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)
default_time

array([11.78379005, 21.38635827,  0.56136058, ...,  4.80147652,
       22.56794342,  3.76740517])

Solution:

In [9]:
# Put your code here to price the corporate bond

def corp_bond(F=1, c=0, def_rate=0.1, rf_rate=0.03, recovery=0.3, mat=10, n_sample=10000):
    U = np.random.uniform(size=n_sample)
    default_time = -(1/def_rate)*np.log(U)
    disc_rate_in = np.exp(-rf_rate*mat)
    disc_rate_out = np.exp(-rf_rate*default_time)
    price = np.where(default_time>mat, 
                    F*disc_rate_in+F*c*disc_rate_in, F*disc_rate_out*recovery)
    return np.mean(price)
# Call your function
corp_bond(F, c, def_rate, rf_rate, recovery, mat)

# Find the mean and std by calling the function 100 times.
n =100
a = [0]*100  
#  if I want to use:
# for i in range(0,100): 
#     corp_bond()
# this will do function 100 times but how to store the results?
while n>=0:
    for i in range(0,100):
        a[i] =corp_bond()
        n = n-1
print('mean',np.mean(a))  # why print('mean',np.mean(a), /n  'std_Dev', np.std(a)) doesn't work?
print( 'std_Dev', np.std(a))

mean 0.44047188477136023
std_Dev 0.002078034974592645


In [256]:
# Put your code here to price the corporate bond

def corp_bond(F=1, c=0, def_rate=0.1, rf_rate=0.03, recovery=0.3, mat=10, n_sample=10000):
    U = np.random.uniform(size = n_sample)
    default_time = -(1/def_rate)*np.log(U)
    disc_rate_in = np.exp(-rf_rate*mat)
    disc_rate_out = np.exp(-rf_rate*default_time)
    price = [0]*10000
    for i in range(0, len(default_time)):
        if default_time[i] > mat:
            price[i] = F*disc_rate_in+F*c*disc_rate_in
        else:
            price[i] = F*disc_rate_out*recovery
    return  np.mean(np.mean(price))
   
# Call your function
corp_bond(F, c, def_rate, rf_rate, recovery, mat)

# Find the mean and std by calling the function 100 times. 
# this method will take much more time.
n =100
a = [0]*100
while n>=0:
    for i in range(1,100):
        a[i] =corp_bond()
        n = n-1
print('mean', np.mean(a))
print('std_Dev', np.std(a))

mean 0.4144670640003538
std_Dev 0.04171940191641513


### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [5]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(9.999999999999998, 10.0)

In [221]:
np.concatenate?

In [258]:
# Now include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(F=1, c= 0, mat=10, def_rate=0.1, rf_rate=0.03, recovery=0.3, n_sample=10000, antithetic=True, mean_match=True):
    U = np.random.uniform(size=n_sample)
    if(antithetic):
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
    else:
        default_time = -(1/def_rate)*np.log(U)
    if(mean_match):
        default_time += 1/def_rate-default_time.mean()
    disc_rate_in = np.exp(-rf_rate*mat)
    disc_rate_out = np.exp(-rf_rate*default_time)
    price = np.where(default_time>mat, 
                     F*disc_rate_in+F*c*disc_rate_in, F*disc_rate_out*recovery)
    return np.mean(price)


# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both
n =100
a = [[0]*100 ,[0]*100 ,[0]*100]
while n>=0:
    for i in range(0,100):
        a[0][i] =corp_bond_cv(antithetic = True, mean_match = False)
        a[1][i] = corp_bond_cv(antithetic = False, mean_match = True)
        a[2][i] = corp_bond_cv(antithetic = True, mean_match = True)
        n = n-1
print('mean', np.mean(a, axis = 1))
print('std_Dev', np.std(a, axis = 1))

mean [0.44021897 0.44028645 0.44039224]
std_Dev [0.00109966 0.00171552 0.00103929]


From the results above we can see that with antithetic random numbers and control variance method, the std_deviation is less then without.

### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [7]:
### Put the analytic expression for the corporate bond price

In [26]:
def corp_bond(F=1, c=0, def_rate=0.1, rf_rate=0.03, recovery=0.3, mat=10):
    prob = 1 - np.exp((-rf_rate-def_rate)*mat)
    price = (F*recovery*def_rate*prob/(rf_rate+def_rate)+(1-prob)*(F+c*F))
    return np.mean(price)
             

In [27]:
corp_bond()

0.440409071564625